# This notebook will tune the models developed in the baseline modeling notebook. It reconstructs a data processing pipeline from an earlier notebook, engineers a "loan-to-value" feature, joins "bureau.csv" via SQLite, then trains a logistic regression and random forest classifier model and compares model performances. Once this process is completed, the engineered dataset and champion model will represent the production level model that will be recreated in source python later.

## Goals for this notebook:
- Implement preprocessing pipeline developed in earlier notebooks
- Feature engineer a loan-to-value column using "AMT_CREDIT" and "AMT_GOODS_PRICE"
- Integrate bureau.csv (aggregate) with SQLite3
- Model the new dataset with logistic regression and random forest classifiers
- Develop insights on the ML models using metrics determined in a previous notebook
- Choose a champion model for finalized pipeline

In [2]:
import pandas as pd
from pathlib import Path
import numpy as np
import sqlite3
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier

## Data Loading, Integration, and Feature Engineering

In [3]:
# Opening application_train.csv
root = Path.cwd().parent

path = root / "data" / "interim" / "application_train.csv"

df_app = pd.read_csv(path)

In [4]:
# Opening bureau.csv
root = Path.cwd().parent

path = root / "data" / "raw" / "raw_dataset" / "bureau.csv"

df_bureau = pd.read_csv(path)

In [5]:
# Connect to SQLite database
db_path = Path("data/interim/home_credit.db")

db_path.parent.mkdir(parents=True, exist_ok=True)

conn = sqlite3.connect(db_path)

db_path.resolve()

PosixPath('/home/tarchy_alls/D502_capstone/notebooks/data/interim/home_credit.db')

In [6]:
# Loading dataframes into SQLite
df_app.to_sql("application", conn, if_exists="replace", index=False)

df_bureau.to_sql("bureau", conn, if_exists="replace", index=False)

1716428

In [7]:
# Aggregating bureau.csv to be grouped on SK_ID_CURR
query = """
SELECT
    SK_ID_CURR,
    COUNT(*) AS bureau_loan_count,
    AVG(CREDIT_DAY_OVERDUE) AS avg_days_overdue,
    MAX(CREDIT_DAY_OVERDUE) AS max_days_overdue
FROM bureau
GROUP BY SK_ID_CURR
"""

bureau_agg = pd.read_sql_query(query, conn)

In [8]:
bureau_agg.head()

,SK_ID_CURR,bureau_loan_count,avg_days_overdue,max_days_overdue
0,100001,7,0.0,0
1,100002,8,0.0,0
2,100003,4,0.0,0
3,100004,2,0.0,0
4,100005,3,0.0,0


In [9]:
# Join bureau aggregated to application dataframe
df = df_app.merge(bureau_agg, on="SK_ID_CURR", how="left")

In [10]:
# Checking if dataframes joined successfully, should be 125 columns (122 from application_train and 3 from bureau_agg)
df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,bureau_loan_count,avg_days_overdue,max_days_overdue
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0.0,0.0,0.0,0.0,0.0,1.0,8.0,0.0,0.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [11]:
# Close connection
conn.close()

In [13]:
# Engineering Loan-to-Value ("LTV") column
ltv = df["AMT_CREDIT"] / df["AMT_GOODS_PRICE"]

df["LTV"] = ltv

df.shape

(307511, 126)

## Data processing

In [17]:
# Determining the aggregated missingness of columns in the dataframe
missing = df.isna().mean()

In [18]:
# Creating an index of which columns to drop from the data frame (thse with more than 50% missing data)
cols_to_drop = missing > 0.5

In [19]:
# Dropping the columns using indexing
df = df.loc[:, ~cols_to_drop].copy()

In [20]:
# Replacing sentinel values ('365243') in DAYS_EMPLOYED with NaN values
df["DAYS_EMPLOYED"] = df["DAYS_EMPLOYED"].replace(365243, np.nan)

In [21]:
# Setting up the dataframe to be split into training and validation dataframes. y-variable is "TARGET", as it is the variable indicating a default within an account
# y is stratified to ensure the default rate is generally similar in the training and validation datasets

df= df.copy()

X = df.drop(columns=["TARGET"])
y = df["TARGET"]

X_train, X_valid, y_train, y_valid = train_test_split(
    X, y,
    test_size = 0.2,
    stratify = y,
    random_state = 69
)

In [22]:
# Indexing numeric columns from the X-dataframes to ensuring imputation is only applied to numeric columns
numeric_cols = X_train.select_dtypes(include=[np.number]).columns

In [23]:
# Utilizing SKlearn SimpleImputer to impute NaN values in numeric columns only
imputer = SimpleImputer(strategy = 'median')

X_train = X_train.copy()
X_valid = X_valid.copy()

X_train[numeric_cols] = imputer.fit_transform(X_train[numeric_cols])
X_valid[numeric_cols] = imputer.transform(X_valid[numeric_cols])

In [24]:
# Indexing categorical columns from the X-dataframes to ensure One-Hot encoding is only applied to categorical columns
categorical_cols = X_train.select_dtypes(include=["object"]).columns

In [25]:
# Defining the OneHotEncoder
one_hot_encoding = OneHotEncoder(
    handle_unknown = "ignore",
    sparse_output = False
)

In [26]:
# Applying One Hot encoding to the X-dataframes
X_train_cat = one_hot_encoding.fit_transform(X_train[categorical_cols])
X_valid_cat = one_hot_encoding.transform(X_valid[categorical_cols])

In [27]:
# Filtering the X-dataframes to select only numeric columns to be combined with the One Hot encoded categorical rows
X_train_num = X_train[numeric_cols].to_numpy()
X_valid_num = X_valid[numeric_cols].to_numpy()

In [28]:
# Combining the numeric and categorical rows back into two now imputed and One Hot encoded X-dataframes
X_train_final = np.hstack([X_train_num, X_train_cat])
X_valid_final = np.hstack([X_valid_num, X_valid_cat])

In [29]:
# Reassigning dataset name for clarity
X_train = X_train_final
X_valid = X_valid_final

## Training models with engineered dataset 

In [30]:
# Scaling datasets for use with Logistic Regression
scaler = StandardScaler()
X_train_num_scaled = scaler.fit_transform(X_train_num)
X_valid_num_scaled = scaler.transform(X_valid_num)

X_train_scaled = np.hstack([X_train_num_scaled, X_train_cat])
X_valid_scaled = np.hstack([X_valid_num_scaled, X_valid_cat])

# Initializing logistic regression
model_lr = LogisticRegression(
    max_iter = 3000,
    class_weight="balanced"
)
model_lr.fit(X_train_scaled, y_train)

,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",'balanced'
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :ter

In [31]:
y_valid_proba_lr = model_lr.predict_proba(X_valid_scaled)[:,1]

In [32]:
# Setting decision threshold
threshold = 0.5
y_pred = (y_valid_proba_lr >= threshold).astype(int)

In [33]:
# Determining confusion matrix to calculate precision & recall
cm = confusion_matrix(y_valid, y_pred)
print(cm)

[[38999 17539]
 [ 1680  3285]]


In [34]:
#  Calculating recall
recall = 3285 / (3285 + 1680)
print(f"The model caught {recall * 100:.1f}% of defaulters")

The model caught 66.2% of defaulters


In [35]:
# Calculating precision
precision = 3215 / (3215 + 17539)
print(f"The model flags {precision*100:.1f}% that actually default")

The model flags 15.5% that actually default


In [36]:
# Getting list of feature names. Using .get_feature_names_out to restore original categorical column names.
num_features = list(numeric_cols)
cat_features = list(one_hot_encoding.get_feature_names_out(categorical_cols))

feature_names = num_features + cat_features

In [37]:
# Generating feature importance ascending
coefs = model_lr.coef_[0]

importance_df_lr = pd.DataFrame({
    "feature": feature_names,
    "Coefficient": coefs,
    "abs_coefficient": np.abs(coefs)
}).sort_values("abs_coefficient", ascending = False)

In [38]:
importance_df_lr.head(20)

,feature,Coefficient,abs_coefficient
91,NAME_INCOME_TYPE_Pensioner,-1.003753,1.003753
175,ORGANIZATION_TYPE_Realtor,0.987125,0.987125
94,NAME_INCOME_TYPE_Unemployed,0.928411,0.928411
96,NAME_EDUCATION_TYPE_Academic degree,-0.892714,0.892714
156,ORGANIZATION_TYPE_Industry: type 12,-0.621916,0.621916
168,ORGANIZATION_TYPE_Legal Services,0.592965,0.592965
193,ORGANIZATION_TYPE_Transport: type 3,0.535319,0.535319
176,ORGANIZATION_TYPE_Religion,-0.515278,0.515278
191,ORGANIZATION_TYPE_Transport: type 1,-0.500979,0.500979
171,ORGANIZATION_TYPE_Mobile,0.492498,0.492498


In [74]:
engineered_list = ["LTV", "bureau_loan_count", "avg_days_overdue", "max_days_overdue"]

importance_df_lr[importance_df_lr["feature"].isin(engineered_list)].sort_values(
    "abs_coefficient", ascending=False
)

,feature,Coefficient,abs_coefficient
70,LTV,0.121755,0.121755
67,bureau_loan_count,0.015639,0.015639
68,avg_days_overdue,0.003199,0.003199
69,max_days_overdue,-0.000289,0.000289


## Logistic Regression Recap
Incorporating simple LTV and bureau aggregates did not improve performance and slightly degraded both precision and recall. This suggests the baseline application features, especially EXT_SOURCE variables, already capture most of the credit risk signal. Therefore, the final model, if we stick with logistic regression, should not be engineered into the dataset.

In [39]:
model_clf = RandomForestClassifier(
    class_weight="balanced",
    min_samples_leaf = 50,
    random_state = 69)

model_clf.fit(X_train, y_train)

,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",100
,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.Note: This parameter is tree-specific.",'gini'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",50
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=""sqrt""The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to `""sqrt""`.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",'sqrt'
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the current node, ``N_t_L`` is the number of samples in theleft child, and ``N_t_R`` is the number of samples in the right child.``N``, ``N_t``, ``N_t_R`` and ``N_t_L`` all refer to the weighted sum,if ``sample_weight`` is passed... versionadded:: 0.19",0.0
,"bootstrap bootstrap: bool, default=TrueWhether bootstrap samples are used when building trees. If False, thewhole dataset is used to build each tree.",True
,"oob_score oob_score: bool or callable, default=FalseWhether to use out-of-bag samples to estimate the generalization score.By default, :func:`~sklearn.metrics.accuracy_score` is used.Provide a callable with signature `metri

In [40]:
y_valid_proba_clf = model_clf.predict_proba(X_valid)[:,1]

threshold = 0.5
y_pred_clf = (y_valid_proba_clf >= threshold).astype(int)

confusion_matrix(y_valid, y_pred_clf)
print(classification_report(y_valid, y_pred_clf))

              precision    recall  f1-score   support

           0       0.95      0.81      0.87     56538
           1       0.19      0.52      0.28      4965

    accuracy                           0.78     61503
   macro avg       0.57      0.66      0.58     61503
weighted avg       0.89      0.78      0.82     61503



In [41]:
importances = model_clf.feature_importances_

importance_df_clf = pd.DataFrame({
    "feature": feature_names,
    "importance": importances
}).sort_values("importance", ascending = False)

importance_df_clf.head(20)

,feature,importance
27,EXT_SOURCE_2,0.152465
28,EXT_SOURCE_3,0.141184
8,DAYS_EMPLOYED,0.050974
7,DAYS_BIRTH,0.044825
70,LTV,0.037032
40,DAYS_LAST_PHONE_CHANGE,0.036141
10,DAYS_ID_PUBLISH,0.032496
3,AMT_CREDIT,0.031958
5,AMT_GOODS_PRICE,0.031357
4,AMT_ANNUITY,0.030158


## Random Forest Classifier recap
Incorporating LTV and bureau aggregates did not change very much in the random forest classifier. Interestingly, RFC seems to utilize more of the engineered features in risk calculation, but the performance of the model changed so little (1% better recall) that the additional noise from feature engineering largely does not impact risk calculation. Additionally, the random forest classifier model still performed worse at catching defaulters. 

# Model Tuning Conclusions
## Notebook Summary:
To recap, in this notebook we aggregated useful columns from "bureau.csv" and integrated them with "application.csv" using SQLite3, then we engineered a loan-to-value ("LTV") column using "AMT_CREDIT" and "AMT_GOODS_PRICE" to create a new dataset to train models with. Then, we reconstructed the data processing pipeline, trained two ML models (Logistic Regression and Random Forest Classifier) using the processed data, and evaluated performance with relevant metrics including precision & recall, capture and lift rate, and feature importance.

## Model Performance on engineered dataset
The logistic regression model performed slightly worse on the new dataset and the random forest classifier performed slightly better on the new dataset. In either case, the performance was so marginal that the additional step of feature engineering is not necessary. More specifically, since the best performing model was logistic regression *without* the feature engineered dataset, this step will be skipped in further productions.

## Insights
While feature engineering didn't improve model performance, it did corroborate that professional credit scores ("EXT_SOURCE_X") already covered the necessary information content of bureau history in the dataset. Additionally, the loan-to-value feature was not weighted very heavily in the best performing model (logistic regression) because the relationship between "AMT_CREDIT" and "AMT_GOODS_PRICE" already represent the "LTV" columns purpose in the base dataset for a linear model, so LTV being present added no new information for the model to use.

## Conclusions
Existing external credit reporting is already covered in the base dataset and the linear nature of the champion model does not necessitate adding a loan-to-vale column. As a result, the final pipeline will retain the simpler baseline feature set both for performance and stability.